In [1]:
from datasets import load_dataset
from transformers import AutoImageProcessor, AutoModelForImageClassification, pipeline
import torch
import PIL

2023-11-20 12:27:56.417209: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-20 12:27:56.435208: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-20 12:27:56.435221: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-20 12:27:56.435233: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-20 12:27:56.438943: I tensorflow/core/platform/cpu_feature_g

In [2]:
processor = AutoImageProcessor.from_pretrained('facebook/dinov2-giant-imagenet1k-1-layer')
model = AutoModelForImageClassification.from_pretrained('facebook/dinov2-giant-imagenet1k-1-layer')
model = model.to("cuda")

In [3]:
imagenet = load_dataset("imagenet-1k", split="train[:128]")

In [4]:
def extract_dinov2_features(sample):
    img = sample['image']
    if (img.mode == 'L') | (img.mode == 'CMYK') | (img.mode == 'RGBA'):
        rgbimg = PIL.Image.new("RGB", img.size)
        rgbimg.paste(img)
        img = rgbimg
    x = torch.tensor(processor(img)['pixel_values'][0]).unsqueeze(0).to("cuda")
    y = model.dinov2.forward(x)[0]
    sample['cls_token'] = y[:, 0].detach()
    sample['patch_tokens'] = y[:, 1:].detach()
    return sample

In [ ]:
imagenet = imagenet.map(extract_dinov2_features)
imagenet = imagenet.remove_columns("image")
imagenet.push_to_hub("danjacobellis/imagenet_dino_dry",split="train")